# Glassdoor Reviews Tech Market

Structure of the Analysis:

1. PySpark **environment setup**
2. Data source and **Spark data abstraction** (DataFrame) **set up**
3. Data set **metadata analysis**:
  1. Display **schema and size** of the DataFrame
  2. Get one or multiple **random samples** from the data set to better understand what the data is all about
4. Remove **NULL values**
    1. Shape filtered dataFrame
5. Understanding the dataset. Reviews per Year
6. **TECH JOB MARKET ANALYSIS**
  1. Different types of job
     1. The most popular cities among data experts
     2. The most popular jobs
     3. Jobs with highest overall satisfaction
     4. Total number of ML / Data Analyst with reviews
     5. Overall satisfaction among DS and Data Analyst compared to the rest
  2. Key players
      1. Firms with more current employees with reviews
      2. Overall satisfaction by firms and current employees
        1. Top-performing firms
        2. The worst firms
      3. Companies with more reviews by current employees in 2019
  9. Understanding Amazon
      1. Cons
      2. Pros




This dataset was scrapped from the Glassdoor portal by Andre Sionek. It contains data for all countries containing the following terms:

* data-scientist
* software-engineer
* data-analyst
* research-scientist
* business-analyst
* product-manager
* project-manager
* data-engineer
* statistician
* dba
* database-engineer
* machine-learning-engineer

Dataset available on Kaggle: https://www.kaggle.com/andresionek/data-jobs-listings-glassdoor?select=glassdoor_reviews.csv

LINKT TO THE VIDEO EXPLAINING THE PROJECT:https://youtu.be/bBKK-WoZIvk

## 1. PySpark environment setup

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql.session import SparkSession
#spark = SparkSession.builder.getOrCreate()

In [3]:
spark = SparkSession.builder\
            .appName("DataFrames Glassdoor")\
            .getOrCreate()

## 2. Data source and Spark data abstraction (DataFrame) setup

In [4]:
reviewsDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("glassdoor_reviews.csv") \
                 .cache() # optimization to make the processing faster



## 3. Data set metadata analysis
### A. Display schema and size of the DataFrame

In [5]:
from IPython.display import display, Markdown
reviewsDF.printSchema()
display(Markdown("ReviewsDF has **%d rows**." % reviewsDF.count()))

root
 |-- id: string (nullable = true)
 |-- index: string (nullable = true)
 |-- reviews.val.cons: string (nullable = true)
 |-- reviews.val.date: string (nullable = true)
 |-- reviews.val.featured: string (nullable = true)
 |-- reviews.val.helpfulCount: string (nullable = true)
 |-- reviews.val.id: string (nullable = true)
 |-- reviews.val.pros: string (nullable = true)
 |-- reviews.val.publishedOn: string (nullable = true)
 |-- reviews.val.publisher: string (nullable = true)
 |-- reviews.val.reviewRatings.careerOpportunities: string (nullable = true)
 |-- reviews.val.reviewRatings.compBenefits: string (nullable = true)
 |-- reviews.val.reviewRatings.cultureValues: string (nullable = true)
 |-- reviews.val.reviewRatings.overall: string (nullable = true)
 |-- reviews.val.reviewRatings.seniorManagement: string (nullable = true)
 |-- reviews.val.reviewRatings.worklifeBalance: string (nullable = true)
 |-- reviews.val.reviewerDuration: string (nullable = true)
 |-- reviews.val.reviewerInf

ReviewsDF has **946168 rows**.

### Selecting only relevant variables for our analysis

In [6]:
from pyspark.sql.functions import col
import pyspark.sql.functions as F
reviewsDF = reviewsDF.select("id",
                             "index",
                             (col("`reviews.val.cons`").alias("cons")),
                            (col("`reviews.val.date`").alias("date")),
                             (col("`reviews.val.pros`").alias("pros")),
                             (col("`reviews.val.reviewRatings.careerOpportunities`").alias("r_opportunities")),
                             (col("`reviews.val.reviewRatings.compBenefits`").alias("r_benefits")),
                             (col("`reviews.val.reviewRatings.cultureValues`").alias("r_culture")),
                             (col("`reviews.val.reviewRatings.overall`").alias("r_overall")),
                             (col("`reviews.val.reviewRatings.seniorManagement`").alias("r_management")),
                             (col("`reviews.val.reviewRatings.worklifeBalance`").alias("r_balance")),
                             (col("`reviews.val.reviewerDuration`").alias("time_as_employee")),
                             (col("`reviews.val.reviewerJobTitle`").alias("job_title")),
                             (col("`reviews.val.reviewerStatus`").alias("Current_Employee")),
                             (col("`reviews.val.reviewerLocation`").alias("location")),
                             (col("`reviews.val.reviewerInformation`").alias("info")),
                             (col("`reviews.val.summaryPoints.recommend`").alias("recommend"))
                             )


### A. Display Schema & Size

In [7]:
reviewsDF.printSchema()
display(Markdown("ReviewsDF has **%d rows**." % reviewsDF.count()))

root
 |-- id: string (nullable = true)
 |-- index: string (nullable = true)
 |-- cons: string (nullable = true)
 |-- date: string (nullable = true)
 |-- pros: string (nullable = true)
 |-- r_opportunities: string (nullable = true)
 |-- r_benefits: string (nullable = true)
 |-- r_culture: string (nullable = true)
 |-- r_overall: string (nullable = true)
 |-- r_management: string (nullable = true)
 |-- r_balance: string (nullable = true)
 |-- time_as_employee: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- Current_Employee: string (nullable = true)
 |-- location: string (nullable = true)
 |-- info: string (nullable = true)
 |-- recommend: string (nullable = true)



ReviewsDF has **946168 rows**.

In [8]:
reviewsDF.sample(False, 0.00001,33).show()

+--------------------+--------------------+--------------------+------------+--------------------+--------------------+----------+--------------+---------------+------------+---------+--------------------+--------------------+----------------+--------------------+--------------------+---------+
|                  id|               index|                cons|        date|                pros|     r_opportunities|r_benefits|     r_culture|      r_overall|r_management|r_balance|    time_as_employee|           job_title|Current_Employee|            location|                info|recommend|
+--------------------+--------------------+--------------------+------------+--------------------+--------------------+----------+--------------+---------------+------------+---------+--------------------+--------------------+----------------+--------------------+--------------------+---------+
|               99762|                null|                null|        null|                null|              

#### We see that there are many rows with only null values. We will create a temporal view to filter for results where id and index are not NULL

## 4. Remove NULL Values

#### Crating a teporal view to use Spark SQL

In [9]:
reviewsDF.createOrReplaceTempView("reviews_v")

In [10]:
filtered =spark.sql("""SELECT * FROM reviews_v 
                    WHERE index IS NOT NULL AND date IS NOT NULL AND id IS NOT NULL
                    AND r_overall<6""")

In [11]:
filtered.createOrReplaceTempView("filtered_v")

In [12]:
spark.sql("""SELECT count(*)  FROM filtered_v""").show()

+--------+
|count(1)|
+--------+
|  236639|
+--------+



#### We end up with 236639 rows from the original 946168 rows

In [13]:
null_rows= ((946168-236639)/946168)

print("Percentage of rows with null values =", (round(null_rows*100,2)))

Percentage of rows with null values = 74.99


#### Almost 3 out of 4 rows from the dataset were rows with null values

### A. Shape filtered dataFrame

In [14]:
print((filtered.count(), len(reviewsDF.columns))) 

(236639, 17)


## 5. Undesrstanding the dataset. Number of reviews per year

In [15]:
spark.sql("""
        SELECT RIGHT(date,4) Year, count(*) number_of_reviews_per_year
        FROM filtered_v 
        WHERE RIGHT(date,4) IN ( "2020","2019","2018","2017","2016","2015","2014","2013","2012", 
                                "2011","2010","2009","2008", "2007","2006")
        GROUP BY Year
        ORDER BY number_of_reviews_per_year DESC
        """).show()

+----+--------------------------+
|Year|number_of_reviews_per_year|
+----+--------------------------+
|2019|                     86104|
|2018|                     35574|
|2017|                     19002|
|2016|                     10866|
|2015|                      6195|
|2014|                      3121|
|2013|                      1640|
|2012|                      1499|
|2011|                       283|
|2010|                       219|
|2009|                       100|
|2008|                        78|
+----+--------------------------+



#### Reviews range from 2008 to 2019 and its number has been increasing over the years

# 6.TECH JOB MARKET ANALYSIS

## Exploring the different types of jobs

### A. What are the most popular cities among data experts?

In [16]:
spark.sql("""
        SELECT location, COUNT(location) 
        FROM filtered_v
        WHERE location IS NOT NULL
        GROUP BY location
        ORDER BY count(*) DESC
        LIMIT 5""").show(5, False)

+----------------------------+---------------+
|location                    |count(location)|
+----------------------------+---------------+
|Singapore (Singapore)       |9316           |
|Shanghai, Shanghai (China)  |5894           |
|Bengaluru (India)           |5867           |
|Paris (France)              |4939           |
|Dublin, Co. Dublin (Ireland)|4732           |
+----------------------------+---------------+



### B. What are the most popular jobs?

In [17]:

spark.sql("""
        SELECT job_title, COUNT(job_title) number_of_professionals
        FROM filtered_v
        WHERE job_title IS NOT NULL AND job_title NOT IN ('Current Employee', 'Former Employee')
        GROUP BY job_title
        ORDER BY count(*) DESC
        LIMIT 10""").show(10,False)

+------------------------+-----------------------+
|job_title               |number_of_professionals|
+------------------------+-----------------------+
|Software Engineer       |7417                   |
|Senior Software Engineer|3234                   |
|Intern                  |3008                   |
|Manager                 |2851                   |
|Project Manager         |2623                   |
|Consultant              |2363                   |
|Software Developer      |2242                   |
|Business Analyst        |2121                   |
|Product Manager         |1855                   |
|Analyst                 |1524                   |
+------------------------+-----------------------+



### C. Jobs with highest overall satisfaction

In [18]:
spark.sql("""
        SELECT job_title, AVG(r_overall) Overall_Satisfaction
        FROM filtered_v
        WHERE job_title IS NOT NULL AND job_title NOT IN ('Current Employee', 'Former Employee')
        GROUP BY job_title
        HAVING COUNT(*)> 200
        ORDER BY Overall_Satisfaction DESC
        LIMIT 10""").show(10,False)

+-----------------------------+--------------------+
|job_title                    |Overall_Satisfaction|
+-----------------------------+--------------------+
|Cloud Support Associate      |5.0                 |
|Human Resources Analyst      |4.904458598726115   |
|Business Manager             |4.820754716981132   |
|Talent Acquisition           |4.772151898734177   |
|Software Engineer(Internship)|4.649079754601227   |
|Mechanical Engineer          |4.581560283687943   |
|Summer Analyst               |4.533519553072626   |
|Summer Intern                |4.46078431372549    |
|Talent Acquisition Specialist|4.438931297709924   |
|Operations Manager           |4.403846153846154   |
+-----------------------------+--------------------+



### D. What is the total number of ML / data analyst

In [19]:
data_analysts = spark.sql("""SELECT *
                FROM filtered_v WHERE 
                                    lower(job_title) LIKE "%data%" OR
                                    lower(job_title) LIKE "%lower(ML)%" OR
                                    lower(job_title) LIKE "%analyst%" OR
                                    lower(job_title) LIKE "%lower(Machine Learning)%"
                                    """)
data_analysts.createOrReplaceTempView("data_analysts_v")

In [20]:
spark.sql("""SELECT count(*) AS number_data_experts
                FROM data_analysts_v
                                    """).show()



+-------------------+
|number_data_experts|
+-------------------+
|              19208|
+-------------------+



### E. What is the average Overall satisfaction among DS/Data Analyst vs the rest?

In [21]:
spark.sql("""SELECT ROUND(AVG(r_overall),2) AS Overall_Rating
                FROM filtered_v""").show()

spark.sql("""SELECT ROUND(AVG(r_overall),2) AS Overall_Data_Analyst
                FROM data_analysts_v""").show()

+--------------+
|Overall_Rating|
+--------------+
|          3.88|
+--------------+

+--------------------+
|Overall_Data_Analyst|
+--------------------+
|                4.06|
+--------------------+



Data Scientists and Data Analysts tend to have an ovarall higher satisfaction

## Exploring the firms

### F. Firms with more current employees with reviews

#### Firms with more current employees who reviewed on Glassdoor

In [22]:
spark.sql("""
        SELECT *
        FROM filtered_v
        WHERE info IS NOT NULL AND Current_Employee=="Current Employee"  """).count()

135211

In [23]:
from pyspark.sql.functions import trim

In [24]:

spark.sql("""
        SELECT LEFT(LTRIM("Current Employyee: I have been working at ", info),6) Name_firm, COUNT(info) Employees_Reviewed, 
        ROUND((Count(info)/135211)*100,2) Percentage_Out_of_total
        FROM filtered_v
        WHERE info IS NOT NULL AND Current_Employee=="Current Employee"  
        AND LEFT(LTRIM("Current Employyee: I have been working at ", info),5) NOT LIKE '%ful%'
        GROUP BY Name_firm
        ORDER BY count(*) DESC
        LIMIT 5""").show()

+---------+------------------+-----------------------+
|Name_firm|Employees_Reviewed|Percentage_Out_of_total|
+---------+------------------+-----------------------+
|   Amazon|              4586|                   3.39|
|   Oracle|              2128|                   1.57|
|   Dell T|              2114|                   1.56|
|   Micros|              1224|                   0.91|
|   Google|               944|                    0.7|
+---------+------------------+-----------------------+



#### Companies with most reviews per current employee in year 2019

In [25]:
spark.sql("""
        SELECT LEFT(LTRIM("Current Employyee: I have been working at ", info),8) Name_firm, COUNT(RIGHT(date,4)) N_reviews_in_2019_by_current_employees
        FROM filtered_v
        WHERE info IS NOT NULL AND Current_Employee=="Current Employee" 
        AND LEFT(LTRIM("Current Employyee: I have been working at ", info),5) NOT LIKE '%ful%' AND RIGHT(date,4)='2019'
        GROUP BY Name_firm
        ORDER BY N_reviews_in_2019_by_current_employees DESC
        LIMIT 10""").show()

+---------+--------------------------------------+
|Name_firm|N_reviews_in_2019_by_current_employees|
+---------+--------------------------------------+
| Amazon f|                                  1443|
| Dell Tec|                                   857|
| Oracle f|                                   692|
| Microsof|                                   501|
| SAP full|                                   390|
| BM full-|                                   381|
| Google f|                                   355|
| J.P. Mor|                                   323|
| PwC full|                                   316|
| DXC Tech|                                   307|
+---------+--------------------------------------+



### G. Overall satisfaction by firms and current employees

#### Top-performing firms

In [26]:

spark.sql("""
        SELECT LEFT(LTRIM("Current Employyee: I have been working at ", info),8) Name_firm, 
        ROUND(AVG(r_overall),2) Overall_rating, ROUND(AVG(r_balance),2) WorkLifeBalance, 
        ROUND(AVG(r_benefits),2) Benefits,ROUND(AVG(r_opportunities),2) Opportunities
        FROM filtered_v
        WHERE info IS NOT NULL AND Current_Employee=="Current Employee" 
        AND LEFT(LTRIM("Current Employyee: I have been working at ", info),5) NOT LIKE '%ful%'
        GROUP BY Name_firm
        HAVING COUNT(*)>100
        ORDER BY Overall_rating DESC
        LIMIT 10""").show()

+---------+--------------+---------------+--------+-------------+
|Name_firm|Overall_rating|WorkLifeBalance|Benefits|Opportunities|
+---------+--------------+---------------+--------+-------------+
| Spotify |          4.99|            4.0|     4.5|         4.48|
| AirAsia |          4.92|           4.26|    4.89|         4.88|
| Zalando |          4.89|            4.9|    4.39|         4.42|
| Hays as |          4.88|           3.79|    4.27|         4.71|
| Red Hat |          4.83|           4.64|    3.82|         4.41|
| Henkel f|          4.81|           4.16|    4.37|         4.73|
| Sabre fu|          4.77|           3.35|    4.26|         4.23|
| Facebook|          4.74|           3.76|     4.3|         4.13|
| Goldman |          4.71|           3.58|    3.93|         4.29|
| DataArt |          4.71|           4.58|    4.47|         4.51|
+---------+--------------+---------------+--------+-------------+



#### The worst firms

In [27]:
spark.sql("""
        SELECT LEFT(LTRIM("Current Employyee: I have been working at ", info),9) Name_firm, 
        ROUND(AVG(r_overall),2) Overall_rating, ROUND(AVG(r_balance),2) WorkLifeBalance, 
        ROUND(AVG(r_benefits),2) Benefits,ROUND(AVG(r_opportunities),2) Opportunities
        FROM filtered_v
        WHERE info IS NOT NULL AND Current_Employee=="Current Employee" 
        AND LEFT(LTRIM("Current Employyee: I have been working at ", info),5) NOT LIKE '%ful%'
        GROUP BY Name_firm
        HAVING COUNT(*)>100
        ORDER BY Overall_rating
        LIMIT 10""").show()

+---------+--------------+---------------+--------+-------------+
|Name_firm|Overall_rating|WorkLifeBalance|Benefits|Opportunities|
+---------+--------------+---------------+--------+-------------+
|Sqlink Gr|           2.0|            2.0|     2.0|          1.0|
|Mitra Int|          2.52|           2.52|     2.5|         1.52|
|Dyson ful|          2.87|           3.08|    2.21|         2.53|
|Tiki.vn f|           3.0|            3.0|     2.0|          3.0|
|TE Connec|          3.03|           2.78|    2.96|          2.8|
|Motorola |          3.21|           3.15|    3.17|         3.41|
|Petrofac |          3.36|           3.06|     3.9|         3.07|
|Hong Kong|          3.41|           3.24|    2.97|         2.74|
|ctronic A|          3.41|           4.06|    3.15|         2.38|
|Takeda Ph|          3.42|           3.16|    3.66|         3.06|
+---------+--------------+---------------+--------+-------------+



## Understanding Amazon

### H. Analyzing the most common reviews for data experts at Amazon

In [28]:
from pyspark.sql.functions import substring_index

In [29]:
reviewsDF2=reviewsDF.where(col("info").like("%Amazon%"))

In [30]:
from pyspark.sql.functions import col, explode, regexp_replace, split


### Cons at Amazon

In [34]:
df =reviewsDF2.select(explode(split(col("cons")," "))).groupBy("col").count()

df.createOrReplaceTempView("df")

spark.sql("""
            SELECT col cons, count FROM df 
            WHERE col NOT IN ("to","the","and", "is","to","of","work", "a","not","you","The",
            "in","are","that", "have","to","can","for", "be","on", "great", "I", "company", "it",
            "as","with","at", "very","this","the","no","so","more", "your", "some", " ")
            ORDER BY count DESC LIMIT 10""").show()

+-------+-----+
|   cons|count|
+-------+-----+
|     No|  588|
|   time|  558|
|   much|  529|
|   life|  440|
|  often|  413|
|balance|  413|
|  learn|  399|
|    too|  372|
|  Cloud|  353|
|   take|  349|
+-------+-----+



### Pros at Amazon

In [32]:
df =reviewsDF2.select(explode(split(col("pros")," "))).groupBy("col").count()

df.createOrReplaceTempView("df")

spark.sql("""
            SELECT col pros, count FROM df 
            WHERE col NOT IN ("to","the","and", "is","to","of","work", "a","not","you","The",
            "in","are","that", "have","to","can","for", "be","on","I", "company", "it",
            "as","with","at", "very","this","the","no", "much","so","more", "your", "some", " ")
            ORDER BY count DESC LIMIT 10""").show()

+-------------+-----+
|         pros|count|
+-------------+-----+
|        Great| 1527|
|  environment|  995|
|        great|  970|
|       people|  965|
|             |  964|
|      working|  927|
|       Amazon|  901|
|opportunities|  817|
|         fast|  736|
|        learn|  734|
+-------------+-----+

